#  Abstractive Summarization of Long Podcast Transcripts with BART using Semantic Self-segmentation
Podcasts are a rapidly growing medium for news, commentary, entertainment, and learning.  Some podcast shows release new episodes on a regular schedule (daily, weekly, etc); others irregularly.  Some podcast shows feature short episodes of 5 minutes or less touching on one or two topics; others may release 3+ hour long episodes touching on a wide range of topics.  Some are structured as news delivery, some as conversations, some as storytelling.

Given a podcast episode, its audio, and transcription, return a short text snippet capturing the most important information in the content. Returned summaries should be grammatical, standalone statement of significantly shorter length than the input episode description.

The user task is to provide a short text summary that the user might read when deciding whether to listen to a podcast. Thus the summary should accurately convey the content of the podcast, and be short enough to quickly read on a smartphone screen. It should also be human-readable.

For further information about the challenge, take a look to Podcasts Track Guidelines:
- [TREC 2020 Podcasts Track](https://trecpodcasts.github.io/participant-instructions-2020.html)

## 1. Dataset preparation

### 1.1 Dataset reading

### 1.2 Dataset analysis

### 1.3 Dataset cleaning

In order to select a subset of the corpus that is suitable for training supervised models, we filtered the descriptions using three heuristics shown in the table below. These filters overlap to some extent, and remove about a third of the entire set. The remaining 66,245 descriptions we call the Brass Set.

| Criterion                        | Threshold                                                    |
| -------------------------------- | ------------------------------------------------------------ |
| Length                           | descriptions that are very long (> 750 characters) or short (< 20 characters) amounting to 24033 or 23% of the descriptions. |
| Similarity to show description   | descriptions with high lexical overlap (over 40%) with their show description, amounting to 9444 or 9% of the descriptions. |
| Similarity to other descriptions | descriptions with high lexical overlap (over 50%) with other episode descriptions amounting 15375 or 15% of the descriptions. |


#### 1.3.1 Further cleaning of the data
The podcast episodes should be restricted to the English language, but they cover a range of geographical regions and we found a number of non-English podcasts in the dataset. So we remove them.

Tokens that correspond to URLs, email addresses, @mentions, #hashtags, and those excessively long tokens (>25 characters) are directly removed from the summaries.

We futher cleaned the descriptions computing a *salience score* for each sentence of the description by summing over word IDF scores.
Then we remove sentences if their salience scores are lower than a threshold. 

### 1.4 Golden set integration
In addition we have a **golden set** of 150 episodes composed by 6 set of summaries for each episode (900 document-summary-grade triplets) that were graded on the Bad/Fair/Good/Excellent scale (0-3). We integrated the best summary of each episodes in the *brass set*.

### 1.5 Semantic segmentation

### 1.6  Chunck classifier

## 2. Training

## 3. Evaluation